<a href="https://colab.research.google.com/github/OpetherMB/grand-debat/blob/main/clean_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ce notebook ne servent que pour faire des traitement sur les dataframes qui va nous servir aprés pour le topic modelling ( voir topic modelling.ipynb)

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from pprint import pprint
!python -m spacy download fr_core_news_md

import fr_core_news_md


nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import  word_tokenize

# spacy for lemmatization
import spacy
!python -m spacy download fr


import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

!python -m spacy download fr_core_news_md
!python -m spacy download fr_core_news_sm
import fr_core_news_md
nlp_fr = fr_core_news_md.load()


import re
import os
pd.set_option('max_colwidth', 400)

     |████████████████████████████████| 88.6MB 1.2MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338490 sha256=ecef8ef49ec6703997ee17af8599ec514566c2c94f865d90a145bdb91c7fd296
  Stored in directory: /tmp/pip-ephem-wheel-cache-omwflvrh/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


<a class="anchor" id="2"></a> 

## 2. Prepare data

In [ ]:
envr_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/grand debat/LA_TRANSITION_ECOLOGIQUE.csv.xz', low_memory=False)
envr_df.head()

,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,authorZipCode,QUXVlc3Rpb246MTYw - Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?,QUXVlc3Rpb246MTYx - Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?,QUXVlc3Rpb246MTQ2 - Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?,"QUXVlc3Rpb246MTQ3 - Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?","QUXVlc3Rpb246MTQ4 - À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?","QUXVlc3Rpb246MTQ5 - Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?","QUXVlc3Rpb246MTUw - Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?",QUXVlc3Rpb246MTUx - Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?,"QUXVlc3Rpb246MTUy - Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","QUXVlc3Rpb246MTUz - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?","QUXVlc3Rpb246MTU0 - Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?","QUXVlc3Rpb246MTU1 - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?","QUXVlc3Rpb246MjA3 - Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?",QUXVlc3Rpb246MTU3 - Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,QUXVlc3Rpb246MTU4 - Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?,QUXVlc3Rpb246MTU5 - Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?
0,UHJvcG9zYWw6ZmZlN2MwNzktMWUyMC0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-4,transition écologique,2019-01-22 09:37:49,2019-01-22 09:37:49,NaN,False,NaN,VXNlcjoxMTQwMTc0YS0xZTFmLTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,97231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Enseignement du tri sélectif à l'école,Multiplier les centrales géothermiques
1,UHJvcG9zYWw6M2RmZmUxZjMtMWUyMS0xMWU5LTk0ZDItZmExNjNlZWIxMWUx,2-5,La surpopulation,2019-01-22 09:39:33,2019-01-22 09:39:33,NaN,False,NaN,VXNlcjpjOWYxZWQ1NS0xYzEwLTExZTktOTRkMi1mYTE2M2VlYjExZTE=,Citoyen / Citoyenne,57000,"Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040. Avec les progrès de la communication dans ce village mon...","Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France ne sont-ils pas dus, avant tout, à la surpopulation mondiale ? Cette population est passée d’1,5 milliards d’habitants en 1900 à 7 milliards en 2020 et montera bientôt à 10 milliards vers 2040. Avec les progrès de la communication dans ce village mon...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mettre en oeuvre au niveau national ses engagements internationaux,"Les problèmes auxquels se trouve confronté l’ensemble de la planète et que dénoncent, dans le plus parfait désordre, les gilets jaunes de France n

In [ ]:
# Change column names
envr_df.columns= ['id', 'reference', 'title', 'createdAt', 'publishedAt', 'updatedAt',
       'trashed', 'trashedStatus', 'authorId', 'authorType', 'authorZipCode', 'problemes', 'reponse_probleme', 'q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16']

# Remove the nan values
envr_df.fillna('',inplace=True)

In [ ]:
# We convert our data to type string
envr_df[['problems', 'reponse_probleme', 'q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16']] = envr_df[['problemes', 'reponse_probleme', 'q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q13','q14','q15','q16']].astype(str)

<a class="anchor" id="3"></a> 

## 3. Define useful processing functions


In [ ]:
def clean(X):
    # X, column data
    pd_list = list(X)
    pd_list = [it.lower() for it in pd_list] # to lowercase
    # We noticed that people use o instead of ô frequently so we replaced it to get more robust text
    pd_list = [re.sub(r'ô','o',it) for it in pd_list] 
    # Same thing with é ê è contributers often misspell these characters so we replaced it with simple e
    pd_list = [re.sub(r'[éêè]','e',it) for it in pd_list] 
    # Delete non alphabetic characters
    pd_list = [re.sub(r'([^a-zéèûôàêô])',' ',it) for it in pd_list]
    return pd_list

In [ ]:
def tokenization(X_list):
    tokenized_list = [word_tokenize(it) for it in X_list]
    return tokenized_list

In [ ]:

#We added this list to give more sens to our results.
other_stopwords = ['les','a','afin','alors',  'plus', 'moins', 'faut', 'tout', 'tous', 'prends', 'tre', 'si','non','doit','avoir','comme','trop','leurs','faire','ils','peut','bien','aussi','cela','gens', 'sans', 'car', 'très', 'fait', '\'', 'qu\'', 'd\'', 'l\'', '"', 'être', 'mai', 'faudrait']

stopWords = stopwords.words('french')
stopWords.extend(other_stopwords)
stopWords = set(stopWords)

def remove_stopwords(X_list):
    filtered_list =[]
    for it in X_list:
        
        temp = []
        for word in it:
            if(word not in stopWords):
                temp.append(word)
        filtered_list.append(temp)
        temp=[]
     
    return filtered_list

In [ ]:
### Function that cleans and removes stopwords
def clean_remove_stopwords(column_data):
  
    non_stopwords_data = remove_stopwords(tokenization(clean(column_data)))
    non_tokenized = []
    for l in non_stopwords_data:
        non_tokenized.append(' '.join(l))
  
    return non_tokenized



In [ ]:
# Function that lemmatized data. We used Spacy 


def lemmetize(column_data):
    rep_lemmas = []
    for i, reponse in enumerate(column_data):
        if(i % 25000 == 0) :
            print('{} processed colmuns now '.format(i))
        if type(reponse) == str:
            doc = nlp_fr(reponse)
 
            lemmas = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN' ,'ADJ']]
        else:
            lemmas = ' '
        rep_lemmas.append(' '.join(lemmas))
    return rep_lemmas


In [ ]:
## test lemmatization
text = ['je suis transitions écologiques', 'une éxpérience écologie naturelle trasformation']
print(lemmetize(text))

0 processed colmuns now 
['écologique', 'éxpérience écologie naturel trasformation']


<a class="anchor" id="5"></a> 

## 5. Save clean dataframes

In [ ]:
clean_df = pd.DataFrame()
clean_df['authorZipCode'] = envr_df['authorZipCode']
clean_column = clean_remove_stopwords(envr_df['title'])
clean_df['title'] = clean_column
clean_column = clean_remove_stopwords(envr_df['problems'])
clean_df['problems'] = clean_column
clean_column = clean_remove_stopwords(envr_df['reponse_probleme'])
clean_df['reponse_probleme'] = clean_column

clean_df.head()

,authorZipCode,title,problems,reponse_probleme
0,97231,transition ecologique,,
1,57000,surpopulation,problemes auxquels trouve confronte ensemble planete denoncent parfait desordre gilets jaunes france dus avant surpopulation mondiale cette population passee milliards habitants milliards montera bientot milliards vers progres communication village mondial chaque individu fin fond asie fin fond afrique passant quartiers campagnes pays aspire vivre peu bl mer mal lotis concitoyens logement nour...,problemes auxquels trouve confronte ensemble planete denoncent parfait desordre gilets jaunes france dus avant surpopulation mondiale cette population passee milliards habitants milliards montera bientot milliards vers progres communication village mondial planete chaque individu fin fond asie fin fond afrique passant quartiers campagnes pays aspire vivre peu bl mer mal lotis concitoyens logem...
2,34140,climat,dereglements climatiques crue secheresse,eviter inondations obliger riverains communes nettoyer secteur
3,17400,pollution air eau,pollution air,taxer gros pollueurs entreprises bateaux marchands routiers etc
4,35430,economie vs ecologie,biodiversite disparition certaines especes,changer mode vie impulser nouvelle economie agricole


In [ ]:
clean_df.to_csv('/content/drive/MyDrive/Colab Notebooks/grand debat/clean_environement.csv')

## 6. Save lemmatized dataframes

In [ ]:
lemmetized_df = pd.DataFrame()
lemmetized_df['authorZipCode'] = clean_df['authorZipCode']
lemmetized_df['title'] = lemmetize(clean_df['title'])
lemmetized_df['problems'] = lemmetize(clean_df['problems'])
lemmetized_df['reponse_probleme'] = lemmetize(clean_df['reponse_probleme'])


lemmetized_df.head()

0 processed colmuns now 
25000 processed colmuns now 
50000 processed colmuns now 
75000 processed colmuns now 
0 processed colmuns now 
25000 processed colmuns now 
50000 processed colmuns now 
75000 processed colmuns now 
0 processed colmuns now 
25000 processed colmuns now 
50000 processed colmuns now 
75000 processed colmuns now 


,authorZipCode,title,problems,reponse_probleme
0,97231,transition ecologiqu,,
1,57000,surpopulation,probleme trouve planet denoncent gilet jaune france du surpopulation mondial population milliard habitant milliard bientot milliard progre communication village mondial individu fin asie fin fond afriqu quartiers campagne pays bl mer mal bien consommation mer probleme tel cas solution probleme stabilisation croissanc demographique richesse partag terre partag eau protection biodiversite confli...,probleme trouve planet denoncent gilet jaune france du surpopulation mondial population milliard habitant milliard bientot milliard progre communication village mondial planet individu fin asie fin fond afriqu quartiers campagne pays bl mer mal bien consommation mer probleme tel cas solution probleme stabilisation croissanc demographique richesse partag terre partag eau protection biodiversite...
2,34140,climat,dereglement climatique cru secheresse,inondation riverain commune secteur
3,17400,pollution eau,pollution air,gros pollueur entreprise bateal marchand routier
4,35430,ecologie,biodiversite disparition espece,mode vie impulser nouveau economie


In [ ]:
lemmetized_df.to_csv('/content/drive/MyDrive/Colab Notebooks/grand debat/lemmatized_environement.csv')